# Common imports & library functions

In [6]:
import collections
from collections import defaultdict, Counter
from dataclasses import dataclass
import doctest
import functools
import itertools
from itertools import count
import math
import re
from copy import deepcopy

# Day 11: Seating System

In [7]:
def _floors(n):
    return ['.'] * n

class SeatLayout:
    def __init__(self, initial_state, neighbor_threshold=4):
        self._neighbor_threshold = neighbor_threshold
        initial_state = initial_state.strip().replace(' ', '')
        cells = [list(row.strip()) for row in initial_state.split()]
        w = len(cells[0])
        self._cells = ([_floors(w + 2)] +
                       [_floors(1) + row + _floors(1) for row in cells] +
                       [_floors(w + 2)])

    def at(self, x, y):
        return self._cells[y+1][x+1]

    def set(self, x, y, state):
        self._cells[y+1][x+1] = state

    def cells(self):
        for y in range(1, len(self._cells) - 1):
            for x in range(1, len(self._cells[0]) - 1):
                yield (x - 1, y - 1, self._cells[y][x])

    def neighbors(self, x, y):
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0: continue
                yield self.at(x+dx, y+dy)

    def update(self):
        prev_layout = deepcopy(self)
        updated = False
        for x, y, cell in prev_layout.cells():
            neighbors = list(prev_layout.neighbors(x, y))
            if (cell == 'L' and not any(n == '#' for n in neighbors)):
                self.set(x, y, '#')
                updated = True
            elif (cell == '#' and neighbors.count('#') >= self._neighbor_threshold):
                self.set(x, y, 'L')
                updated = True
        return updated

    def num_occupied(self):
        return sum(r.count('#') for r in self._cells)

    def __str__(self):
        return '\n'.join(''.join(r) for r in self._cells)

class SeatLayout2(SeatLayout):
    def __init__(self, initial_state, neighbor_threshold=5):
        super().__init__(initial_state, neighbor_threshold)

    def within_grid(self, x, y):
        return 0 <= y < len(self._cells) - 1 and 0 <= x < len(self._cells[0]) - 1

    def neighbors(self, x, y):
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0: continue
                for xp, yp in zip(count(x+dx, dx), count(y+dy, dy)):
                    if not self.within_grid(xp, yp):
                        break
                    if (cell := self.at(xp, yp)) != '.':
                        yield cell
                        break

def simulate_until_equilibrium(layout):
    while layout.update():
        continue
    return layout.num_occupied()

In [8]:
test_layout = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

assert simulate_until_equilibrium(SeatLayout(test_layout)) == 37
assert simulate_until_equilibrium(SeatLayout2(test_layout)) == 26

In [57]:
# Final answers
with open('day11.txt') as f:
    initial_state = f.read()
    print('Part 1: ', simulate_until_equilibrium(SeatLayout(initial_state)))
    print('Part 2: ', simulate_until_equilibrium(SeatLayout2(initial_state)))

Part 1:  2418
Part 2:  2144
